In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
try:
    df=pd.read_csv("C:/Users/Administrator/Downloads/Dataset .csv",encoding='utf-8')
    print('File loaded sucessfully!')
except FileNotFoundError:
    print("Error: 'Dataset.csv' not found. Please ensure the CSV file is in the same directory.")
    exit()

File loaded sucessfully!


In [48]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,��stanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz A��ac۱,208,��stanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,A���k Kahve,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


In [49]:
df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [50]:
df['Cuisines'][0]

'French, Japanese, Desserts'

In [51]:
df[['Restaurant Name']]

,Restaurant Name
0,Le Petit Souffle
1,Izakaya Kikufuji
2,Heat - Edsa Shangri-La
3,Ooma
4,Sambo Kojin
...,...
9546,Naml۱ Gurme
9547,Ceviz A��ac۱
9548,Huqqa
9549,A���k Kahve


In [52]:
df['Restaurant Name'][43]

'Confeitaria Colombo'

In [53]:
df[['Cuisines']]

,Cuisines
0,"French, Japanese, Desserts"
1,Japanese
2,"Seafood, Asian, Filipino, Indian"
3,"Japanese, Sushi"
4,"Japanese, Korean"
...,...
9546,Turkish
9547,"World Cuisine, Patisserie, Cafe"
9548,"Italian, World Cuisine"
9549,Restaurant Cafe


In [54]:
import re

In [55]:
df['Restaurant Name']=df['Restaurant Name'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x))
df['Cuisines']=df['Cuisines'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x))
df['City']=df['City'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x))

In [56]:
indices=pd.Series(df.index,index=df['Restaurant Name'].str.lower()).drop_duplicates()
print(indices)

Restaurant Name
le petit souffle              0
izakaya kikufuji              1
heat - edsa shangri-la        2
ooma                          3
sambo kojin                   4
                           ... 
naml gurme                 9546
ceviz aac                  9547
huqqa                      9548
ak kahve                   9549
walters coffee roastery    9550
Length: 9551, dtype: int64


In [57]:
df['combined_features']=(
    df['Aggregate rating'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x)) + " " +
    df['Cuisines'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x)) + " " +
    df['City'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x)) + " " +
    df['Locality'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x)) + " " +
    df['Price range'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x)) + " " +
    df['Restaurant Name'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x)) + " " +
    df['Votes'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z\s,-]','',x))
)

In [58]:
df[['combined_features']]

,combined_features
0,"French, Japanese, Desserts Makati City Centur..."
1,"Japanese Makati City Little Tokyo, Legaspi Vi..."
2,"Seafood, Asian, Filipino, Indian Mandaluyong ..."
3,"Japanese, Sushi Mandaluyong City SM Megamall,..."
4,"Japanese, Korean Mandaluyong City SM Megamall..."
...,...
9546,Turkish stanbul Karaky Naml Gurme
9547,"World Cuisine, Patisserie, Cafe stanbul Kouyo..."
9548,"Italian, World Cuisine stanbul Kurueme Huqqa"
9549,Restaurant Cafe stanbul Kurueme Ak Kahve


In [59]:
df['combined_features']=df['combined_features'].apply(lambda x : x.lower())

In [60]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf_vectorizer.fit_transform(df['combined_features'])

In [91]:
print(tfidf_matrix)

  (0, 5665)	0.2991766580521341
  (0, 4659)	0.27728186295775514
  (0, 3474)	0.20876685250942517
  (0, 4762)	0.2991766580521341
  (0, 3707)	0.1194857611752133
  (0, 1089)	0.2991766580521341
  (0, 1352)	0.42823748941789347
  (0, 3688)	0.5727380479218979
  (0, 1690)	0.11623795785431834
  (0, 2902)	0.16566149511676453
  (0, 2166)	0.2106254775278696
  (1, 3234)	0.3231487026029882
  (1, 2843)	0.3231487026029882
  (1, 6371)	0.21133738732337076
  (1, 3487)	0.3231487026029882
  (1, 6112)	0.2758503961470488
  (1, 3534)	0.23697458923031978
  (1, 1352)	0.3083671701135599
  (1, 3688)	0.6186296695816346
  (1, 2902)	0.17893540748397924
  (2, 2588)	0.24879533329298292
  (2, 4419)	0.22472604541865968
  (2, 3389)	0.35359692036878126
  (2, 5463)	0.4247598130246786
  (2, 1900)	0.49759066658596585
  :	:
  (9547, 1094)	0.4308785191743021
  (9547, 3320)	0.4308785191743021
  (9547, 5732)	0.339214961559353
  (9547, 4582)	0.349366384016792
  (9547, 6541)	0.29814099142190403
  (9547, 1529)	0.30922403938118626
  (

In [62]:
print(f' TF-IDF matrix created with {tfidf_matrix.shape[0]} documents and {tfidf_matrix.shape[1]} features ')

 TF-IDF matrix created with 9551 documents and 6665 features 


In [63]:
def get_recommendations(user_pref,df_data,tfidf_vec,tfidf_mat,num_recomm=5):
    user_prefs=re.sub(r'[^a-zA-Z0-9\s]', '',user_pref.lower())
    user_tfidf=tfidf_vec.transform([user_prefs])
    cosine_sim=cosine_similarity(user_tfidf,tfidf_mat).flatten()

    restaurant_indices=cosine_sim.argsort()[-num_recomm:][::-1]

    recommend_restaurants=df_data.iloc[restaurant_indices]

    recommend_restaurants['Similarity_Score']=[cosine_sim[idx] for idx in restaurant_indices]

    return recommend_restaurants[['Restaurant Name','Cuisines','Locality','City','Price range','Aggregate rating','Votes','Similarity_Score']]

In [64]:
print('\n -- Testing the recommendation system with user preferences -- ')
user_prefs_1 = "Italian cuisine in Connaught Place, high rating"
print(f' Recommendations for : {user_prefs_1}')
recommendations_1 = get_recommendations(user_prefs_1, df, tfidf_vectorizer, tfidf_matrix, num_recomm=5)
recommendations_1


 -- Testing the recommendation system with user preferences -- 
 Recommendations for : Italian cuisine in Connaught Place, high rating


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19084\1206173204.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_restaurants['Similarity_Score']=[cosine_sim[idx] for idx in restaurant_indices]


,Restaurant Name,Cuisines,Locality,City,Price range,Aggregate rating,Votes,Similarity_Score
6677,High Street Caf,"North Indian, Continental, Chinese, Italian",Rajouri Garden,New Delhi,3,3.8,374,0.391198
3009,My Bar Square,"North Indian, Chinese, Italian, Continental",Connaught Place,New Delhi,3,2.5,1096,0.377343
3008,My Bar Lounge Restaurant,"North Indian, Chinese, Italian, Continental",Connaught Place,New Delhi,3,2.7,2460,0.370321
3069,Sbarro,"Pizza, Italian",Connaught Place,New Delhi,2,3.5,1311,0.362995
6797,High On Burgers,Fast Food,Safdarjung,New Delhi,1,3.2,37,0.362477


In [65]:
user_prefs_2 = "Cheap burger in London"
print(f"Recommendations for: '{user_prefs_2}'")
recommendations_2 = get_recommendations(user_prefs_2, df, tfidf_vectorizer, tfidf_matrix, num_recomm=5)
recommendations_2

#print("\n" + "="*50 + "\n")



Recommendations for: 'Cheap burger in London'


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19084\1206173204.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_restaurants['Similarity_Score']=[cosine_sim[idx] for idx in restaurant_indices]


,Restaurant Name,Cuisines,Locality,City,Price range,Aggregate rating,Votes,Similarity_Score
9380,Duck Waffle,British,"Bishopsgate, City Of London",London,3,4.9,706,0.508282
6620,London Street Kitchen,"North Indian, Mughlai, Chinese",Rajinder Nagar,New Delhi,2,4.2,163,0.460421
9387,The Breakfast Club,"American, Burger","D'arblay Street, Soho",London,3,4.5,313,0.452558
1874,Burger Point,"Burger, Fast Food",Sector 31,Gurgaon,1,2.7,7,0.428173
1724,Burger Point,"Burger, Fast Food",Sector 14,Gurgaon,1,3.2,20,0.428173


In [66]:
# Example User Preferences 3: Indian food, any location, good reviews
user_prefs_3 = "Indian food, good reviews"
print(f"Recommendations for: '{user_prefs_3}'")
recommendations_3 = get_recommendations(user_prefs_3, df, tfidf_vectorizer, tfidf_matrix, num_recomm=5)
recommendations_3




Recommendations for: 'Indian food, good reviews'


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19084\1206173204.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_restaurants['Similarity_Score']=[cosine_sim[idx] for idx in restaurant_indices]


,Restaurant Name,Cuisines,Locality,City,Price range,Aggregate rating,Votes,Similarity_Score
1328,Good Food,Healthy Food,DLF Phase 2,Gurgaon,1,0.0,0,0.705760
3284,Good Foods,"North Indian, Fast Food, Chinese",Delhi University-GTB Nagar,New Delhi,1,3.3,35,0.597901
1177,Good to Go,Raw Meats,Ardee City,Gurgaon,2,0.0,1,0.539319
5110,Good Luck Cafe,Ice Cream,Majnu ka Tila,New Delhi,1,0.0,0,0.419893
7083,Elmas at Good Earth,"Cafe, Continental, Italian","Select Citywalk Mall, Saket",New Delhi,3,4.0,61,0.383203


In [67]:
print("\nRecommendation system complete. You can now use the `get_recommendations_from_preferences` function to provide personalized recommendations.")


Recommendation system complete. You can now use the `get_recommendations_from_preferences` function to provide personalized recommendations.
